In [1]:
import pandas as pd
import os
import random
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
num_neg_ex = 1

pos_label = "1"
neg_label = "0"

cnn_dailymail_path = "cnn_dailymail"
data_path = "data/cnn_dailymail"

In [3]:
def preprocess_df(df):
    data = []
    
    for i in tqdm(range(len(df.index))):
        article = df['article'][i].lower()
        highlight = df['highlights'][i].lower()
        for j in range(num_neg_ex):
            # Write positive example
            data.append(["[CLS] " + article + " [SEP] " + highlight + " [SEP]", pos_label])
            
            # Write negative examples
            neg_ex_index = random.choice([k for k in range(len(df.index)) if k not in [i]])
            data.append(["[CLS] " + article + " [SEP] " + df['highlights'][neg_ex_index].lower() + " [SEP]", neg_label])
    
    data_df = pd.DataFrame(data, columns=['input', 'label'])
    return data_df

In [ ]:
train_df = pd.read_csv(f"{cnn_dailymail_path}/train.csv")
val_df = pd.read_csv(f"{cnn_dailymail_path}/validation.csv")

train_df = preprocess_df(pd.concat([train_df, val_df], ignore_index=True))

os.makedirs(os.path.dirname(f"{data_path}/train.csv"), exist_ok=True)
train_df.to_csv(f"{data_path}/train.csv", index=False)

  0%|          | 0/300481 [00:00<?, ?it/s]

In [ ]:
test_df = pd.read_csv(f"{cnn_dailymail_path}/test.csv")
test_df = preprocess_df(test_df)
test_df.to_csv(f"{data_path}/test.csv", index=False)